In [1]:
#mount drive to get access to videos and folders
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os
from skimage import color, img_as_float, io
import matplotlib.pyplot as plt

#path to save frames that are significantly different from the previous frame
output_pathpath = '/content/drive/MyDrive/SARA/frames/'

#read the video
path = '/content/drive/MyDrive/SARA/trafficvideoslower.mp4'
vid = cv2.VideoCapture(path)

#check is video is open
if not vid.isOpened():
    print("Error: Could not open video file")
    exit()

prev_frame = None
count=0

output_path = output_pathpath
os.chdir(output_pathpath)

while True:
  ret, frame = vid.read() #reads the next frame of the video with each loop

  if frame is None:
    break #End of video

  if prev_frame is not None: #this will make comparison starts from the second frame

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) #convert previous frame to grayscale
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert current frame to grayscale

    prev_blur = cv2.GaussianBlur(prev_gray, (25, 25), 0) #reduces noise and smooths the image on previous frame
    curr_blur = cv2.GaussianBlur(curr_gray, (25, 25), 0) #reduces noise and smooths the image on current frame

    #calculate the absolute difference between previous and current blured frame
    diff_frame = cv2.absdiff(prev_blur, curr_blur) #the result is an image that highlights the pixels where there is a big difference

    _, threshold_frame = cv2.threshold(diff_frame, 20, 255, cv2.THRESH_BINARY) #creates a binary image where moving areas are white

     # Add these lines to merge nearby regions (car parts)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    threshold_frame = cv2.dilate(threshold_frame, kernel, iterations=2)

    #detect contours in the binary threshold frame
    contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

   #look through all the contours and draw a reclangle around them
    movement_detected = False
    for contour in contours:
        if 4500 < cv2.contourArea(contour) < 100000:
          (x, y, w, h) = cv2.boundingRect(contour)
          cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 2)
          movement_detected = True
    if movement_detected:
        cv2.imwrite(output_path+ "%d.png" %count, frame)#save the frame if it's different
        count+=1

  prev_frame = frame #next frame

vid.release()